In [4]:
from qiskit.circuit import QuantumCircuit, Qubit, Clbit
from qiskit_aer import AerSimulator
import qiskit.quantum_info as qi
import numpy as np

test_cases = [
    ('0.1', '[0.990234375, 0.009765625]'),
    ('0.3124456', '[0.915283203125, 0.084716796875]'),
    ('0.5', '[0.8125, 0.1875]'),
    ('0.577350269', '[0.777778, 0.222222]'),
    ('0.9', '[0.842529296875, 0.157470703125]')
]

def beam_splitter(r):
    """
    Returns the beam splitter matrix.

    Args:
        - r (float): The reflection coefficient of the beam splitter.
    Returns:
        - (np.array): 2 x 2 matrix that represents the beam
        splitter matrix.    
    """
    t = (1 - (r ** 2)) ** (1 / 2)
    matriz_r = np.array([[r, t],
                         [t, (-r)]])
    return matriz_r

def mz_interferometer(r):
    """
    This quantum circuit returns the probability that either A or C
    detect a photon, and the probability that D detects a photon.
    
    Args:
        - r (float): The reflection coefficient of the beam splitters.
    Returns: 
        - np.array(float): An array of shape (2,), where the first 
        element is the probability of detection at A or C,
        and the second element is the probability of detection at D.
    """
    shots = 2**20
    backend = AerSimulator()
    entradas = [Qubit(), Clbit()]
    qc = QuantumCircuit(entradas)
    bc = qi.Operator(beam_splitter(r))
    qc.unitary(bc, [0], label= "Beam_Splitter")
    qc.measure(0, 0)
    with qc.if_test((entradas[1],1)) as else_:
        qc.unitary(bc, [0], label= "Beam_Splitter")
        qc.measure(0, 0)
    backend = AerSimulator()
    job = backend.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    for k, v in counts.items():
        if k == '0':
            probabilidade_A_C = v/shots
        else:
            probabilidade_D = v/shots
    return np.array([probabilidade_A_C,probabilidade_D])

display(mz_interferometer(0.1))
display(mz_interferometer(0.3124456))
display(mz_interferometer(0.5))
display(mz_interferometer(0.577350269))
display(mz_interferometer(0.9))

array([0.99024868, 0.00975132])

array([0.9119606, 0.0880394])

array([0.81253624, 0.18746376])

array([0.77706051, 0.22293949])

array([0.84594917, 0.15405083])